In [ ]:
!pip install accelerate datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.5 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map = "auto")

# input_text = "Write me a poem about Machine Learning."
# input_ids = tokenizer(input_text, return_tensors="pt")

# outputs = model.generate(**input_ids)
# print(tokenizer.decode(outputs[0]))


tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset
math_dataset_arith = load_dataset('math_dataset','arithmetic__add_or_sub')

Generating train split:   0%|          | 0/1999998 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
math_dataset_arith

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 1999998
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 10000
    })
})

In [ ]:
# Select just 20 questions from the training split
subset_dataset = math_dataset_arith["train"].select(range(20))

In [ ]:
for question in subset_dataset["answer"]:
  print(question[1:][i.replace("\n","") for i in data_list])

'-1977321734.94\n'
'436273717\n'
'4062036.93\n'
'18135.801\n'
'30927.427689\n'
'-1348127737.98\n'
'-96508225\n'
'-24311053.3\n'
'-2925.472\n'
'50582662\n'
'118.99324042\n'
'46.9039\n'
'-95280.8\n'
'-93684378\n'
'837.996917\n'
'1704809972.7\n'
'9807982\n'
'63014.937\n'
'-0.315003381\n'
'3258771.6\n'


In [ ]:
['-1977321733.', '436273717', '4061960.', '18135.801', 'number.', '-1348127536', '9.', '-24311054.', '-2836.528', '50582796.', 'number.', '47.2999', '94183.8.', '-93684778', '0.1.', '1704809941.', '9808022.', '62952.', '0.11103405.', '3258774.6.']

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
input_text = "Add together 2 and 436273715.\n Think step by step and answer. Then give only the final answer."
input_ids = tokenizer(input_text, return_tensors="pt").to(device)

outputs = model.generate(**input_ids,max_length=512)
print(tokenizer.decode(outputs[0]))

text = tokenizer.decode(outputs[0])
parts = text.split("=")

# Extract the numerical value after the equal sign
final_answer = parts[-1].split("<")[0].strip()

# Print the final answer
print(final_answer)

<bos>Add together 2 and 436273715.
 Think step by step and answer. Then give only the final answer.

2 + 436273715 = 436273717<eos>
436273717


## Baseline

In [ ]:
answers = []

In [ ]:
len(answers)

1000

In [ ]:

q = 0
for question in subset_dataset["question"]:
  prompt = question[1:] + " Think step by step and answer. Then give only the final answer."
  input_ids = tokenizer(prompt, return_tensors="pt").to(device)
  outputs = model.generate(**input_ids,max_length=512)
  text = tokenizer.decode(outputs[0])
  answers.append(text)
  q +=1


In [ ]:
len(answers)

1000

In [ ]:
# file_name = "output.txt"

with open('output.txt', 'w', encoding='utf-8') as file:
    for item in answers:
        file.write("%s\n" % item)

## Without COD

In [ ]:
for question in subset_dataset["question"][:5]:
  prompt = question[1:] + "Give only the final answer."
  print(prompt)

'Total of 0.06 and -1977321735.\n'Give only the final answer.
'Add together 2 and 436273715.\n'Give only the final answer.
'Calculate 4062037 + -0.07.\n'Give only the final answer.
'What is the difference between -57 and -18192.801?\n'Give only the final answer.
'2.427689 - -30925\n'Give only the final answer.


In [ ]:
answers = []

In [ ]:
q = 0
for question in subset_dataset["question"]:
  prompt = question[1:] + "Give only the final answer."
  input_ids = tokenizer(prompt, return_tensors="pt").to(device)
  outputs = model.generate(**input_ids,max_length=512)
  text = tokenizer.decode(outputs[0])
  answers.append(text)
  q +=1


In [ ]:
answers

["<bos>'Total of 0.06 and -1977321735.\\n'Give only the final answer.\n\nThe total is -1977321735.<eos>"]

In [ ]:
with open('output_withoutcod.txt', 'w', encoding='utf-8') as file:
    for item in answers:
        file.write("%s\n" % item)

## Improving it

In [ ]:
for question in subset_dataset["question"][:5]:
  print(question)

b'Add -2.1786 and 274.107.\n'
b'What is -0.4 - 2128734365?\n'
b'Work out 141511.9 - -39.938.\n'
b'Sum 8 and 1430529053.\n'
b'-0.043135 - 0.037247\n'


In [ ]:
prompt = subset_dataset["question"][0] + " Think step by step and answer. Then give only the final answer."
input_ids = tokenizer(prompt, return_tensors="pt").to(device)
outputs = model.generate(**input_ids,max_length=512)
text = tokenizer.decode(outputs[0])

In [ ]:
text

"<bos>b'Add -2.1786 and 274.107.\\n' Think step by step and answer. Then give only the final answer.\n\n-2.1786 + 274.107 = 251.935.<eos>"

In [ ]:
subset_dataset["question"][0]

"b'Add -2.1786 and 274.107.\\n'"

In [ ]:
b = [answer.split("'")[1].rstrip("\\n") for answer in a]


## Self Conistency

In [ ]:
# subset_dataset["answer"]

["b'-1977321734.94\\n'",
 "b'436273717\\n'",
 "b'4062036.93\\n'",
 "b'18135.801\\n'",
 "b'30927.427689\\n'",
 "b'-1348127737.98\\n'",
 "b'-96508225\\n'",
 "b'-24311053.3\\n'",
 "b'-2925.472\\n'",
 "b'50582662\\n'",
 "b'118.99324042\\n'",
 "b'46.9039\\n'",
 "b'-95280.8\\n'",
 "b'-93684378\\n'",
 "b'837.996917\\n'",
 "b'1704809972.7\\n'",
 "b'9807982\\n'",
 "b'63014.937\\n'",
 "b'-0.315003381\\n'",
 "b'3258771.6\\n'"]

In [ ]:
for answer in subset_dataset["answer"][:2]:
  print(answer)

b'-1977321734.94\n'
b'436273717\n'


In [ ]:
for question in subset_dataset["question"][:2]:
  prompt = question[1:] + "Give only the final answer."
  input_ids = tokenizer(prompt, return_tensors="pt").to(device)
  outputs = model.generate(**input_ids,max_length=512)
  text_0 = tokenizer.decode(outputs[0])

  prompt_1 = "Are you sure that the answer for " + prompt + "is " + text_0 + "?"
  input_ids = tokenizer(prompt_1, return_tensors="pt").to(device)
  outputs = model.generate(**input_ids,max_length=512)
  text_1 = tokenizer.decode(outputs[0])

  prompt_2 = "Are you sure that the answer for " + prompt + "is " + text_1 + "?"
  input_ids = tokenizer(prompt_1, return_tensors="pt").to(device)
  outputs = model.generate(**input_ids,max_length=512)
  text_2 = tokenizer.decode(outputs[0])

  print(text_2)






<bos>Are you sure that the answer for 'Total of 0.06 and -1977321735.\n'Give only the final answer.is <bos>'Total of 0.06 and -1977321735.\n'Give only the final answer.

The total is -1977321735.<eos>?<eos>
<bos>Are you sure that the answer for 'Add together 2 and 436273715.\n'Give only the final answer.is <bos>'Add together 2 and 436273715.\n'Give only the final answer.

2 + 436273715 = 436273717<eos>?<eos>


In [ ]:
prompt = 'Total of 0.06 and -1977321735.\n Give only the final answer.'
input_ids = tokenizer(prompt, return_tensors="pt").to(device)
outputs = model.generate(**input_ids,max_length=512, temperature = 0.1)
text_0 = tokenizer.decode(outputs[0])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [ ]:
text_0

'<bos>Total of 0.06 and -1977321735.\n Give only the final answer.\n\nThe total is -1977321735.<eos>'